<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong> CC3104 - Aprendizaje por Refuerzo 
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Laboratorio 7:</strong> Policy Gradients Methods</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autores:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>Mario Antonio Guerra Morales - <strong>21008</strong></li>
      </ul>
    </td>
  </tr>
</table>

## 📝 Task 1

**Explique la diferencia entre los métodos de aprendizaje de refuerzo basados en valores y en políticas. ¿Por qué los métodos de gradiente de políticas son especialmente útiles para entornos con espacios de acción continua?**

En lo que se refiere a los métodos value-based, ellos aprenden una función de valor y conforme van eligiendo acciones que maximizan dicho valor, también van derivando su política. Mientras que los policy-based aprenden directamente de los parámetros de una política, optimizando así el retorno esperado con la gradiente de la política. Es debido a esto que los métodos de gradiente en políticas son útiles en espacios de acción continua, porque la acción se muestrea de una distribución diferenciable y el entrenamiento ajusta sus parámetros por la gradiente, evitando maximizar explícitamente sobre un espacio continuo.

## 📝 Task 2

**Implemente una versión simple del algoritmo REINFORCE. Use un entorno simple, como CartPole-v1 de OpenAI Gym, para entrenar a un agente. La función de valor estimado debe utilizar un aproximador de función lineal.**

Pasos a considerar:

* Inicialice la política (por ejemplo, una red neuronal o una política softmax).

* Simule episodios y obtenga recompensas.

* Calcule el rendimiento descontado para cada par de estado-acción.

* Actualice los parámetros de la política utilizando actualizaciones de gradiente de política.

* Agregue una línea base (función de valor estimado) para reducir la varianza.

In [ ]:
import random
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

try:
    import gymnasium as gym
    GYMN = True
except Exception:
    import gym
    GYMN = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

@dataclass
class Config:
    env_id: str = "CartPole-v1"
    gamma: float = 0.99
    policy_lr: float = 3e-3
    value_lr: float = 5e-3
    hidden_sizes: tuple = (128,)
    max_episodes: int = 500
    max_steps_per_episode: int = 500
    reward_goal: float = 475.0
    rolling_window: int = 20
    log_every: int = 10

cfg = Config()
cfg

Config(env_id='CartPole-v1', gamma=0.99, policy_lr=0.003, value_lr=0.005, hidden_sizes=(128,), max_episodes=500, max_steps_per_episode=500, reward_goal=475.0, rolling_window=20, log_every=10)

### Inicialización de la política

In [3]:
class PolicyNet(nn.Module):
    # MLP -> acciones. 
    # Política categórica con softmax implícito.
    def __init__(self, obs_dim, act_dim, hidden=(128,)):
        super().__init__()
        layers, d = [], obs_dim
        for h in hidden:
            layers += [nn.Linear(d, h), nn.ReLU()]
            d = h
        layers += [nn.Linear(d, act_dim)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class ValueLinear(nn.Module):
    # Baseline lineal V(s) = w^T s + b
    def __init__(self, obs_dim):
        super().__init__()
        self.v = nn.Linear(obs_dim, 1, bias=True)

    def forward(self, x):
        return self.v(x).squeeze(-1)

def initialize_models_and_optimizers(obs_dim, act_dim, cfg: Config):
    policy = PolicyNet(obs_dim, act_dim, cfg.hidden).to(DEVICE)
    baseline = ValueLinear(obs_dim).to(DEVICE)
    opt_pi = optim.Adam(policy.parameters(), lr=cfg.policy_lr)
    opt_v  = optim.Adam(baseline.parameters(), lr=cfg.value_lr)
    return policy, baseline, opt_pi, opt_v

### Método para simulación de episodios y obtención de recompensas

In [ ]:
@torch.no_grad()
def simulate_episode(env, policy, max_steps, device=DEVICE):
    if GYMN:
        obs, _ = env.reset()
    else:
        obs = env.reset()

    states, actions, logps, rewards = [], [], [], []

    for _ in range(max_steps):
        s_t = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        logits = policy(s_t)
        dist = torch.distributions.Categorical(logits=logits)
        a_t = dist.sample()
        logp_t = dist.log_prob(a_t)

        step = env.step(a_t.item())
        if GYMN:
            obs_next, r, term, trunc, _ = step
            done = term or trunc
        else:
            obs_next, r, done, _ = step

        states.append(s_t.squeeze(0).cpu().numpy())
        actions.append(a_t.item())
        logps.append(logp_t.item())
        rewards.append(float(r))

        obs = obs_next
        if done:
            break

    return states, actions, logps, rewards

### Cálculo del rendimiento descontado para cada estado-acción

In [ ]:
# Código aquí

### Actualización de los parámetros de la política utilizando actualizaciones de gradiente de política

In [ ]:
# Código aquí

### Agregación de línea base/función de valor estimado para reducción de la varianza

In [ ]:
# Código aquí

### Entrenamiento

In [ ]:
# Código aquí